In [20]:
# For Python 2 / 3 compactability
# from __future__ import print_function

# Sample dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# if you want you can add more features & examples.
# Interesting note: 2nd and 5th examples have the same features, but different labels.
# Let's see how tree handles this case.

In [21]:
# Data
training_data = [
    ["Green", 3, "Mango"],
    ["Yellow", 3, "Mango"],
    ["Red", 1, "Grape"],
    ["Red", 1, "Grape"],
    ["Yellow", 3, "Lemon"]
]

In [22]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [23]:
def unique_vals(rows, cols):
    """Find the unique values for a column in a dataset."""
    return set([rows[cols] for row in rows])

In [24]:
##
# Demo:
# unique_vals(training_data, 0)
# unique_vals(training_data 1)
##

In [25]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {} # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = rows[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [26]:
##
# Demo:
# class_count(training_data)
##

In [27]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [28]:
##
# Demo:
# is_numeric(7)
# is_numeric("Red
##

In [29]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        # This is just a helper method to print
        # the question in readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value))
        

In [30]:
def partition(rows, question):
    """Partition a dataset.
    For each row int the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for rows in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [31]:
## 
# Demo:
# Let's partition the training data based on whether rows are Red.
# true_rows, false_rows = partition(training_data, Question(0, "Red"))
# This will contain all the "Red" rows.
# true_rows
# This will contain everything else.
# false_rows
##

def gini(rows):
    """Calculate the Gini impurity for a list of rows.
    There are a few different ways to do this, i thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learningGini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity

In [32]:
##
# Demo:
# Let's look at some example to understand how Gini Impurities works.
##

def info_gain(left, right, current_uncertainity):
    """Information Gain
    The uncertainity of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainity - p * gini(left) - (1 - p) * gini(right)

In [33]:
##
# Demo:
# Calculate the uncertainity of our training data.
# current_uncertainity = gini(trainig_data)
#
# How much information do we gain by partitioning on 'Green'?
# true_rows, false_rows = partition(training_data, Question(0, 'Green'))
# info_gain(true_rows, false_rows, current_uncertainity)
#
# what about if we partitioned on 'Red' instead?
# true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# info_gain(true_rows, false_rows, current_uncertainity)

In [34]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0 # keep track of the best information gain
    best_question = None # keep train of the feature / value that produced it
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    
    for col in range(n_features): # for each feature
        values = set([row[col] for row in rows]) # unique values in the column
        for val in values: # for each value
            question = Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainity)

            # You actually can use '>' instead of '>=' here
            # but i wanted the tree to look a certain way for our toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [35]:
##
# Demo
# find the best question to ask first for our toy dataset.
# best_gain, best_question = find_best_split(training_data)
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
##

class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Mango") -> number of times
    It appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
        
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """
    
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
def build_tree(rows):
    """Builds the tree. 
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    """
    # Try partitioning the dataset on each of the unique attribute.
    # calculate te information gain.
    # and retain the question that produces the highest gain.
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # Since we can ask no further question,
    # We'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true_branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records te best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    # Base case: we' ve reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # Print the question at this node
    print(spacing + str(node.question))
    
    # Call this function recursively or te true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    
def classify(row, node):
    # Base case: we' ve reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true_branch or the false_branch.
    # Compare the feature / value stored in the node,
    # to the example we' re considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [36]:
#
# Demo
# the tree predicts the 1st row of our
# training_data is an apple with confidence 1.
# my_tree = build_tree(training_data)
# classify(training_data[0], my_tree)
#

def print_leaf(counts):
    """Print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

#
# Demo:
# Printing that a bit nicer
# print_leaf(classify(training_data[0], my_tree))
#


In [38]:
#
# Demo:
# On the second example, the confidence is lower
# print_leaf(classify(training_data[1], my_tree))
##

if __name__ == '__main__':
    my_tree = build_tree(training_data)
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ["Green", 3, "Mango"],
        ["Yellow", 4, "Apple"],
        ["Red", 2, "Grape"],
        ["Red", 1, "Grape"],
        ["Yellow", 3, "Lemon"]
    ]
    
    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
             row[-1], print_leaf(classify(row, my_tree)))

TypeError: unhashable type: 'list'

In [ ]:
# Next steps
# - add support for missing (or unseen) attributes
# - prune the tree to prevent overfitting
# - add support for regression